In [1]:
# -*- coding: utf-8 -*-
import os, sys
import json

import glob
sys.path.append("..")
sys.path.append("../..")
import dataset_utils.utils as utils
import dataset_utils.config as config
import cv2
import dlib
import openface

from utils.FaceAligner import FaceAligner

- 顔検出時に複数の結果が出たときに、サイズが最大のものを採用する
- 位置を考慮したほうがいいが、後に回す

In [2]:
size = 96
indices = openface.AlignDlib.OUTER_EYES_AND_NOSE
src_dir = os.path.join(config.DS_DIALOG, "transformed_split")
tgt_dir = os.path.join(config.DS_DIALOG, "aligned_face")
movie_files = ["01_01.MP4", "01_02.MP4", "02_01.MP4", "02_02.MP4", "04_01.MP4", "04_02.MP4", "05_01.MP4", "05_02.MP4", "03_01.MP4", "03_02.MP4"]

json_dir = os.path.join(config.DS_DIALOG, "face_bb_and_landmarks")
aligner = FaceAligner()

In [3]:
def calc_bb_size(bb):
    return (bb[2]-bb[0]) * (bb[3] * bb[1])

def select_largest(bbs):
    sizes = [calc_bb_size(bb['bb']) for bb in bbs]
    max_i  = sizes.index(max(sizes))
    return bbs[max_i]

In [ ]:
for movie_file in movie_files:
    print(movie_file)
    movie_id = movie_file.split(".")[0]
    src_sub_dir = os.path.join(src_dir, movie_id)
    tgt_sub_dir = os.path.join(tgt_dir, movie_id)
    if not os.path.exists(tgt_sub_dir):
        os.makedirs(tgt_sub_dir)
    json_path = os.path.join(json_dir, "%s.json" % movie_id)
    with open(json_path, "r") as fr:
        bblms = json.load(fr)
        
    src_paths = sorted(glob.glob(os.path.join(src_sub_dir, "*.png")))
    for i, path in enumerate(src_paths):
        img_name = os.path.basename(path)
        img = utils.imread(path)
        if img_name in bblms and bblms[img_name]:
            bblm = select_largest(bblms[img_name])
            bb = bblm['bb']
            bb_rec = dlib.rectangle(bb[0], bb[1], bb[2], bb[3])
            lms = bblm['landmarks']
            aligned_img = aligner.align(img=img, bb=bb_rec, landmarks=lms, size=size, indices=indices)
            tgt_path = os.path.join(tgt_sub_dir, img_name)
            utils.imwrite(tgt_path, aligned_img)
    utils.notify("@nogaken align_face (%s) done!!" % movie_id)

utils.notify("@nogaken align_face done!!")

01_01.MP4
